# OpenAI Function Calling in LangChain

Based on [**this tutorial**](https://learn.deeplearning.ai/courses/functions-tools-agents-langchain/lesson/4/openai-function-calling-in-langchain)

# Setup

In [1]:
from dotenv import load_dotenv

In [3]:
_ = load_dotenv()

# Global Imports

In [4]:
from typing import List

from pydantic import BaseModel, Field

# Pydantic Syntax

Pydantic data classes are a blend of Python's data classes **with the validation power of Pydantic**.

They offer a concise way to:
- define data structures,
- ensure that the data adheres to specified types and constraints.

In standard python, you would create a class this way:

In [5]:
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [6]:
foo = User(name="Joe",age=32, email="joe@gmail.com")

In [7]:
foo.name

'Joe'

But, unfortunately, **there's no way to "control" or "enforce" the types, as seen on the next example**:

In [8]:
foo = User(name="Joe",age="bar", email="joe@gmail.com")

In [9]:
foo.age

'bar'

Hopefully, `pydantic` solves this with implemented data validation.

In [10]:
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [11]:
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")

In [12]:
foo_p.name

'Jane'

In [15]:
# pydantic takes care of data validation
try:
    foo_p = pUser(name="Jane", age="bar", email="jane@gmail.com")
except Exception as e:
    print(e)

1 validation error for pUser
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='bar', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/int_parsing


`pydantic` also allows to **nest data structures**.

In [16]:
class Class(BaseModel):
    students: List[pUser]

In [17]:
obj = Class(
    students = [pUser(name="Jane", age=32, email="jane@gmail.com")]
)

In [18]:
obj

Class(students=[pUser(name='Jane', age=32, email='jane@gmail.com')])

# Pydantic to OpenAI Function Definition

In [19]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

**This class will be useful to create a corresponding function**.

In [20]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [21]:
weather_function = convert_pydantic_to_openai_function(WeatherSearch)

In [22]:
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

We get a JSON schema with the appropriate form to pass to OpenAI.

Note that **the docstring within the Class is mandatory, as it becomes the function's description in the output JSON schema**.

In [23]:
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")

In [24]:
try:
    convert_pydantic_to_openai_function(WeatherSearch1)
except Exception as e:
    print(e)

In [25]:
convert_pydantic_to_openai_function(WeatherSearch1)

{'name': 'WeatherSearch1',
 'description': '',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

> **NOTE**
> 
> And so this contradicts what's said in the video, This is now no more mandatory but you get a function with no description, which is really unconvenient for the model.